In [ ]:
'''
Optuna: A hyperparameter optimization framework. uses Bayesian optimization to find the best 
        hyperparameters for machine learning models.
Bayesian Optimization: A technique in which a relation is built between hyperparameters and
        model performance using a probablistic model, where new hyperparameters are chosen
        based on past results to maximize performance(finding the maxima of an unknown function).
Key Terms:
Study: An optimization task, which contains multiple trials.
Trial: A single execution of the objective function with a specific set of hyperparameters.
Trail Parameters: These are specific hyperparameter values chosen during a trial.
                 "The combination of multiple trials becomes a study."
Objective Function: A user-defined function that takes hyperparameters as input and returns
        a performance metric to be optimized (ex: In classification the the objective function
        could be cross-entropy loss, which optuna seeks to minimize).
Sampler: A component that suggests hyperparameter values for each trial based on past results.
Pruner: A component that stops unpromising trials early to save computational resources.
        
     
'''